<a href="https://colab.research.google.com/github/UVE-R/MNIST-Neural-Network/blob/main/Training_New.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!gdown 1O0HV67yQBuxXNUI5p_bKXCT9hz7-BAOD

Downloading...
From: https://drive.google.com/uc?id=1O0HV67yQBuxXNUI5p_bKXCT9hz7-BAOD
To: /content/kaggle.json
100% 62.0/62.0 [00:00<00:00, 185kB/s]


In [2]:
!ls -lha kaggle.json

-rw-r--r-- 1 root root 62 Aug 31 01:13 kaggle.json


In [3]:
!pip install kaggle -q      # At first, I suspect the kaggle API lose effect so it doesn't have .kaggle folder. (not working)
!rm -rf /root/.kaggle      # when I created the folder, it says the file or dir already exits
!mkdir /root/.kaggle        # successful
!mv kaggle.json /root/.kaggle/kaggle.json    # not sure if I have to use full destination path, I previously only used /root/.kaggle and it failed. Don't have time to validate this thought.
!ls /root/.kaggle/kaggle.json

/root/.kaggle/kaggle.json


In [4]:
! kaggle competitions download alaska2-image-steganalysis

100% 30.0G/30.0G [04:56<00:00, 154MB/s]
100% 30.0G/30.0G [04:56<00:00, 109MB/s]


In [5]:
!rm -rf JMiPOD

In [6]:
!unzip alaska2-image-steganalysis

Streaming output truncated to the last 5000 lines.
  inflating: UERD/74675.jpg          
  inflating: UERD/74676.jpg          
  inflating: UERD/74677.jpg          
  inflating: UERD/74678.jpg          
  inflating: UERD/74680.jpg          
  inflating: UERD/74681.jpg          
  inflating: UERD/74682.jpg          
  inflating: UERD/74683.jpg          
  inflating: UERD/74684.jpg          
  inflating: UERD/74685.jpg          
  inflating: UERD/74686.jpg          
  inflating: UERD/74687.jpg          
  inflating: UERD/74688.jpg          
  inflating: UERD/74689.jpg          
  inflating: UERD/74690.jpg          
  inflating: UERD/74691.jpg          
  inflating: UERD/74692.jpg          
  inflating: UERD/74693.jpg          
  inflating: UERD/74694.jpg          
  inflating: UERD/74695.jpg          
  inflating: UERD/74696.jpg          
  inflating: UERD/74697.jpg          
  inflating: UERD/74698.jpg          
  inflating: UERD/74699.jpg          
  inflating: UERD/74700.jpg          

In [7]:
!rm alaska2-image-steganalysis.zip

In [8]:
!rm -rf JMiPOD

In [9]:
!rm -rf Test

In [10]:
!rm -rf UERD

In [11]:
!rm -rf sample_data

In [12]:
!rm -rf sample_submission.csv

In [13]:
!mkdir data

In [14]:
!mkdir data/training

In [15]:
!mkdir data/validation

In [16]:
!mkdir data/training/Cover
!mkdir data/training/JUNIWARD

In [17]:
!mkdir data/validation/Cover
!mkdir data/validation/JUNIWARD

In [18]:
import os
import shutil

def split_data(source_dir, training_dir, validation_dir, split_size):
    files = os.listdir(source_dir)
    training_size = split_size * len(files)

    print("Starting Split")

    for i, file in enumerate(files):
        if (i < training_size):
            shutil.copyfile(os.path.join(source_dir, file), os.path.join(training_dir, file))
        else:
            shutil.copyfile(os.path.join(source_dir, file), os.path.join(validation_dir, file))

In [19]:
cover_source_dir = "Cover"
juni_source_dir = "JUNIWARD"

training_path = "data/training"
validation_path = "data/validation"

training_cover_dir = os.path.join(training_path, "Cover")
validation_cover_dir = os.path.join(validation_path, "Cover")

training_juni_dir = os.path.join(training_path, "JUNIWARD")
validation_juni_dir = os.path.join(validation_path, "JUNIWARD")

split_size = 0.8

In [20]:
if(len(os.listdir(training_cover_dir)) > 0):
    print("Cover training directory is not empty! Aborting splitting")
elif (len(os.listdir(validation_cover_dir)) > 0):
    print("Cover validation directory is not empty! Aborting splitting")
elif (len(os.listdir(training_juni_dir)) > 0):
    print("JUNIWARD training directory is not empty! Aborting splitting")
elif (len(os.listdir(validation_juni_dir)) > 0):
    print("JUNIWARD validation directory is not empty! Aborting splitting")
else:
    split_data(cover_source_dir, training_cover_dir, validation_cover_dir, split_size)
    split_data(juni_source_dir, training_juni_dir, validation_juni_dir, split_size)

Starting Split
Starting Split


In [21]:
print(f"\nCover directory has {len(os.listdir(cover_source_dir))} images")
print(f"Original JUNIWARD directory has {len(os.listdir(juni_source_dir))} images\n")

print(f"There are {len(os.listdir(training_cover_dir))} Cover images for training")
print(f"There are {len(os.listdir(training_juni_dir))} JUNIWARD images for training")

print(f"There are {len(os.listdir(validation_cover_dir))} Cover images for validation")
print(f"There are {len(os.listdir(validation_juni_dir))} JUNIWARD images for validation")


Cover directory has 75000 images
Original JUNIWARD directory has 75000 images

There are 60000 Cover images for training
There are 60000 JUNIWARD images for training
There are 15000 Cover images for validation
There are 15000 JUNIWARD images for validation


In [22]:
from keras import mixed_precision

mixed_precision.set_global_policy('mixed_float16')

The dtype policy mixed_float16 may run slowly because this machine does not have a GPU. Only Nvidia GPUs with compute capability of at least 7.0 run quickly with mixed_float16.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once


In [23]:
training_batch_size = 16
validation_batch_size = 16

In [24]:
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input


train_datagen = ImageDataGenerator(preprocessing_function = preprocess_input)

train_generator = train_datagen.flow_from_directory(
    training_path,
    target_size = (512, 512),
    batch_size = training_batch_size,
    class_mode = "binary")

Found 120000 images belonging to 2 classes.


In [25]:
validation_datagen = ImageDataGenerator(preprocessing_function = preprocess_input)

validation_generator = validation_datagen.flow_from_directory(
    validation_path,
    target_size = (512, 512),
    batch_size = validation_batch_size,
    class_mode = "binary")

Found 30000 images belonging to 2 classes.


In [26]:
import tensorflow as tf
import numpy as np

In [27]:
%pip install efficientnet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 1.3 MB/s eta 0:00:00


In [28]:
import efficientnet.tfkeras as efn

# B7 seems to be the best!
pretrained_model = tf.keras.applications.efficientnet_v2.EfficientNetV2B0(
    include_top = False,
    pooling = "avg",
    classes = 2,
    weights = "imagenet")

for layer in pretrained_model.layers:
    layer.trainable = True


x = pretrained_model.output
output = tf.keras.layers.Dense(1, activation = "sigmoid", dtype = "float32")(x)   # May need dtype = float32

model = tf.keras.Model(inputs=pretrained_model.input, outputs=output)

24274472/24274472 [==============================] - 0s 0us/step


In [29]:
optimizer = tf.keras.optimizers.AdamW(learning_rate = 0.001, weight_decay = 0.01)

In [30]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor = "val_loss",
    patience = 2,
    factor = 0.5
)

In [31]:
model.compile(
    loss="binary_crossentropy",
    optimizer=optimizer,
    metrics=["accuracy"]
)

In [32]:
# Checkpoint to save best model seen so far
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint('model-{epoch:03d}.h5', monitor='val_loss', save_best_only=True, mode='auto')

In [ ]:
EPOCHS = 50

history = model.fit(
    train_generator,
    epochs = EPOCHS,
    validation_data = validation_generator,
    callbacks = [checkpoint, reduce_lr]
)

Epoch 1/50
   3/7500 [..............................] - ETA: 1564:14:05 - loss: 0.6828 - accuracy: 0.5000